In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from scipy import stats
import tensorflow as tf
import math
import random

fps=120
threshold=1
moving_avg_len=10
filter_value = 20
length=200


def readFileData(file):
    column_names = ['wmx1', 'wmy1', 'wmz1']

    data = pd.read_csv(file, skiprows = 1 , names = column_names)
 
    wx = data["wmx1"]
    wy = data["wmy1"]
    wz = data["wmz1"]

    records=movingavg(wx,wy,wz)

    return np.dstack([wx,wy,wz])[0]

def readData(directory):
    records = []
    labels = np.empty(0)
    
    allFiles = glob.glob("*.csv")
    for i,file in enumerate(allFiles):
        fileName = os.path.basename(file)
        print("file name is", fileName,"num is",i)
        (name, ext) = os.path.splitext(fileName)
        #parts = name.split("_")
        if (True):
            label = "circle"
            fileData = readFileData(file)
            print("file datat is ",fileData)
            records.append(fileData)
            labels = np.append(labels, label)

    return (records, labels)

def movingavg(x,y,z):
    x_avg = np.zeros(moving_avg_len)/moving_avg_len
    y_avg = np.zeros(moving_avg_len)/moving_avg_len
    z_avg = np.zeros(moving_avg_len)/moving_avg_len
    x_avg = np.convolve(x,x_avg,'same')
    y_avg = np.convolve(y,y_avg,'same')
    z_avg = np.convolve(z,z_avg,'same')
    
    return [x_avg,y_avg,z_avg]

def splitData(records):##sungshil
    print("records is ",records[0][0])
    term=fps//4
    record_sum=[]
    record=[]
    record_all=[]
    ct=0
    sum_now=0.0
    sum_pre=0.0
    start=0
    end=0
    
    for i in range((len(records[0])//(term))-100):
        sum=0.0
        record_x=np.empty(shape=[1],dtype=float)
        record_y=np.empty(shape=[1],dtype=float)
        record_z=np.empty(shape=[1],dtype=float)
        for j in range(fps):
            record_x=np.append(record_x,records[0][term*i+j][0])
            record_y=np.append(record_y,records[0][term*i+j][1])
            record_z=np.append(record_z,records[0][term*i+j][2])
        for k in range(fps-2):
            sum=sum+(record_x[k]-record_x[k+1])**2+(record_y[k]-record_y[k+1])**2+(record_z[k]-record_z[k+1])**2
            
        sum_pre=sum_now
        sum_now=sum
        record_sum.append(sum)
        if (sum_pre<threshold and sum_now>threshold):
            
            ct=ct+1
            start=term*i
            print("start",start)
        if (sum_pre>threshold and sum_now<threshold):
            end=term*i  
            print("end",end)
            print("step is " ,ct)
            print("length of end-start",(end-start))
            s=[]
            for k in range(end-start):
                #t=(records[0][term*i+j+k])
                #print("t is",t)
                s.append(records[0][start+k])
                #print("s is ",s)
            record_all.append(s)
             
    print("number of patterm : ",ct)
    print("average is",np.average(record_sum))
    print("number of term",len(record_sum))
  
    plt.plot(record_sum[1500:2500])
    
    return record_all

def getRecordsMaxLength(records):
    maxLen = 0
    for record in records:
        if (len(record) > maxLen):
            maxLen = len(record)
        
    return maxLen

def shortenRecordsLen(records, length):
    ret=[]
    rec_len=len(records)
    
    for index in range(len(records)):
        s=[]
        record = records[index]
        if len(record) > length and len(record)<3000:
            for i in range(length):
                r=[]
                t=i*len(record)/length
                if t%1==0:
                    s.append(records[index][int(t)])
                else:
                    a=t//1
                    s1=t-a
                    s2=a+1-t
                    for i in range(3):
                        r.append(s1*records[index][int(a)][i]+s2*records[index][int(a)][i])
                    s.append(r) 
        
                
        ret.append(s)
    plt.plot
    return ret

def weight_variable(shape):#가중치 생성
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):#편향 생성
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):#각 채널마다 활성곱 수행후 ReLu활성화 함수로 pass
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases))
    
def apply_max_pool(x,kernel_size,stride_size):#생성된 convolution층에 pooling 적용
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

C:\Users\sskim\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
(records,labels) = readData("dr")
labels = np.asarray(labels)
print(len(labels))
#print("size of ",len(records[0]))
#print("size of divide ",len(records[0])//40)
#records=filter_data(records)
records=splitData(records)
records=shortenRecordsLen(records, length)
print(records)
print(records[0])

for i in range(len(records)):
    plt.plot(records[i])

0


IndexError: list index out of range

In [ ]:
records = np.asarray(records)
reshaped_records = records.reshape(len(records),1,200, 3) #recoed를 1차원으로 reshape
print(reshaped_records)

In [ ]:
train_test_split = np.random.rand(len(reshaped_records)) < 0.70
train_test_split.astype(np.int)
print(train_test_split)
# training과 testing set 랜덤하게 70/30 나누기
train_x = reshaped_records[train_test_split[0]]
print(train_x)
train_y = labels[train_test_split[0]]
test_x = reshaped_records[~train_test_split[0]]
test_y = labels[~train_test_split[0]]
print(len(train_x))

In [ ]:
input_height = 1
input_width = 200
num_labels = 6
num_channels = 3 #?? input과 channel의 차이 무엇?

batch_size = 10
kernel_size = 60#??
depth = 60
num_hidden = 1000

learning_rate = 0.0001
training_epochs = 8

total_batches = train_x.shape[0] // batch_size

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
# input data
Y = tf.placeholder(tf.float32, shape=[None,num_labels])
# output data

c = apply_depthwise_conv(X,kernel_size,num_channels,depth) #1d convolution
p = apply_max_pool(c,20,2) #1d Max-Pooling
c = apply_depthwise_conv(p,6,depth*num_channels,depth//10) #1d convolution

shape = c.get_shape().as_list()
c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

f_weights_l1 = weight_variable([shape[1] * shape[2] * depth * num_channels * (depth//10), num_hidden])
f_biases_l1 = bias_variable([num_hidden])
f = tf.nn.tanh(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)
# softmax

In [ ]:
loss = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:

cost_history = np.empty(shape=[1],dtype=float)
 
with tf.Session() as session: #training, 위에서 c, p는 어떻게 apply하는 거지?
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)
        print ("Epoch: ",epoch," Training Loss: ",c," Training Accuracy: ",
              session.run(accuracy, feed_dict={X: train_x, Y: train_y}))
    
    print ("Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y}))

